# Machine Learning for disease diagnosis using omics data

This file and raw data in github repo here: https://github.com/parkyed/sepsis_ml_omics_msc


# Code:

In [20]:
import pandas as pd
import numpy as np
import os
from scipy import stats
from collections import OrderedDict
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn.model_selection import KFold, RepeatedKFold
import warnings
from joblib import dump, load
from pickle import dump, load
warnings.filterwarnings(action="ignore", message="^internal gelsd")
warnings.filterwarnings(action="ignore", message="Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.")
#np.random.seed(1)


In [21]:
os.getcwd()

'/Users/Ed/Documents/GitHub/sepsis_ml_omics_msc'

### Data Import

In [22]:
# import and check data

raw_data = pd.read_csv('genomic_data.csv')
print('The number of columns is:  '+str(len(raw_data.columns)))
print('The number of row is:  '+str(len(raw_data)))

The number of columns is:  93
The number of row is:  48804


### Checking for Duplicates

In [11]:
# Counting the number of duplicate rows using various columns, NaN values excluded before looking for duplicates.

dup_column = ['NuID', 'Search_Key', 'ILMN_Gene', 'RefSeq_ID', 'Entrez_Gene_ID', 'Probe_Id']
for column in dup_column:
    data = raw_data.dropna(subset=[column])
    dup_list = []
    for index, value in data.duplicated(subset=[column]).items():
        if value == True:
            dup_list.append(index)
    print(f"# duplicate rows based on {column}:   "+str(len(dup_list)))

# duplicate rows based on NuID:   0
# duplicate rows based on Search_Key:   4671
# duplicate rows based on ILMN_Gene:   11000
# duplicate rows based on RefSeq_ID:   5742
# duplicate rows based on Entrez_Gene_ID:   10897
# duplicate rows based on Probe_Id:   1


In [29]:
raw_data.tail(10)

,NuID,Species,Source,Search_Key,Transcript,ILMN_Gene,Source_Reference_ID,RefSeq_ID,Unigene_ID,Entrez_Gene_ID,...,Inf_149,Inf_152,Inf_157,Inf_159a,Inf_162a,Inf_164,Inf_191,Inf_198,Inf_203,Fold change
48794,ZZQvowA1A0rdICPUkk,Homo sapiens,RefSeq,ILMN_24212,ILMN_24212,CASP8AP2,NM_012115.2,NM_012115.2,NaN,9994.0,...,7.0826,7.1113,7.0518,7.0041,7.0160,6.9972,7.1369,7.0338,7.0494,NaN
48795,ZzTi0SkIUngIr1kHu0,Homo sapiens,RefSeq,ILMN_137958,ILMN_37701,C20ORF7,NM_024120.3,NM_024120.3,NaN,79133.0,...,7.8497,7.7296,7.5873,7.4077,7.4651,7.4088,7.6038,7.5458,7.5341,NaN
48796,ZzTla5Up6pZQklKEU4,Homo sapiens,RefSeq,ILMN_27978,ILMN_27978,TMEM151A,NM_153266.2,NM_153266.2,NaN,256472.0,...,7.0177,6.9288,7.0070,6.9190,6.9754,6.9154,7.0130,7.0644,6.9691,NaN
48797,ZZTZrt5dKGKbmWEeks,Homo sapiens,RefSeq,ILMN_31400,ILMN_31400,LOC645781,XM_933141.2,XM_933141.2,NaN,645781.0,...,7.1248,7.0515,7.0518,7.1918,7.0641,7.0524,6.9758,7.0188,7.0494,NaN
48798,ZztzUERXUExGBETFFU,Homo sapiens,RefSeq,ILMN_14674,ILMN_14674,NPY6R,NR_002713.1,NR_002713.1,NaN,NaN,...,7.0010,7.0157,6.9841,6.9733,7.0813,7.0127,6.9264,7.0644,6.9361,NaN
48799,ZzVW43OPaE6GO1WJqs,Homo sapiens,RefSeq,ILMN_139314,ILMN_139314,SEZ6L2,XM_939114.1,XM_939114.1,NaN,26470.0,...,7.1036,7.0593,7.0298,7.1770,7.1478,7.0254,7.0245,6.9904,6.9919,NaN
48800,ZZXgKFKE5q5rp53.p0,Homo sapiens,RefSeq,ILMN_42654,ILMN_42654,LOC339047,XM_932631.1,XM_932631.1,NaN,339047.0,...,7.0509,7.0305,6.9952,7.0171,7.0033,7.0127,7.1234,7.0338,7.0380,NaN
48801,ZzXXie6icAfD693B6w,Homo sapiens,Unigene,ILMN_104503,ILMN_104503,HS.539415,Hs.539415,NaN,Hs.539415,NaN,...,7.0423,7.0072,7.0298,7.0441,7.0160,7.0254,7.0245,7.1131,7.0494,NaN
48802,ZzZ5OziA4Anaop_fcU,Homo sapiens,RefSeq,ILMN_14103,ILMN_165051,MAGI1,NM_015520.1,NM_015520.1,NaN,9223.0,...,7.0423,7.0593,7.1600,7.0308,7.0294,7.0672,7.0606,7.0644,6.9804,NaN
48803,NaN,Homo sapiens,RefSeq,ILMN_8135,ILMN_8135,TUBB2A,NM_001069.2,NM_001069.2,NaN,7280.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Drop duplicate rows on Probe_Id and transpose

In [83]:
# reformat data, drop duplicates, transpose, index on probe_id, drop columns with all NaN, drop Fold change

# drop duplicate rows based on Probe_Id, and fold change column
df = raw_data.drop_duplicates(subset=['Probe_Id'], keep='first')
df = df.drop(['Fold change'], axis=1)

# split into gene code dataframe, and data set based on unique probes
gene_df = df.iloc[:, 4:15]
df = df.iloc[:, np.r_[14, 29:92]]

# transpose data set and index on probe ID, drop all columns that are all NaN values
df = df.transpose()
df = df.rename(columns=df.iloc[0]).drop(df.index[0])

# identify any columns that are all NaN values, and drop from both data df and gene_df
nan_columns = df.columns[df.isna().any()].tolist()
df = df.drop(nan_columns, axis=1)
gene_df = gene_df[gene_df['Probe_Id'] != nan_columns[0]]
print('Number of features in dataset:   '+str(len(df.columns)))
print('Number of genes in the gene name database:    '+str(len(gene_df.index)))
print('Number of examples in dataset:  '+str(len(df)))
print('Number of Missing values in dataset:   '+str(df.isnull().sum().sum()))
print('The number of features with all zeros is:   '+str(sum((df == 0).all(axis=0))))
print('Check indices of df and gene_df match. sum of matching index:   '+str(sum(df.columns == gene_df['Probe_Id'])))

# create copy of the data without patient Inf_075
df_no75 = df.drop(['Inf075'], axis=0)

Number of features in dataset:   48802
Number of genes in the gene name database:    48802
Number of examples in dataset:  63
Number of Missing values in dataset:   0
The number of features with all zeros is:   0
Check indices of df and gene_df match. sum of matching index:   48802


In [84]:
# create labels for both data sets

labels = []
for item in df.index:
    if 'Con' in item:
        labels.append(0)
    else:
        labels.append(1)
labels = np.asarray(labels)

labels_no75 = []
for item in df.index:
    if item == 'Inf075':
        continue
    if 'Con' in item:
        labels_no75.append(0)
    else:
        labels_no75.append(1)
labels_no75 = np.asarray(labels_no75)

# check correct
#list(zip(df.index, labels))
#list(zip(df_no75.index, labels_no75))

### Standardisation (z-scoring)

Data standardised (i.e. z-scored) for the following reasons:
- L1 regularisation penalties in logistic regressions assumes data centred at zero and on the same scale
- Distance based ML models such as SVM require and assume standardised data, otherwise variables on larger scales disproportionally impact the model
- Am using the output coeffiecients from logistic regression as a crude measure of feature importance, and so in order to compare coefficients as a measure of relative importance, variables must be standardised

Reference: https://towardsdatascience.com/normalization-vs-standardization-quantitative-analysis-a91e8a79cebf

In [88]:
# standardise by imputing NaN values and using standard scale - z-scoring
# note - given imputer not necessary, could move standard scaler into the pipeline for each model

def standardise(examples):
    scaler = StandardScaler()
    examples_scaled = scaler.fit_transform(examples)
    return examples_scaled

X_df = standardise(df)
X_df = pd.DataFrame(X_df, columns=df.columns, index=df.index)
print(X_df.shape)

# dataset excluding patient 75
X_df_no75 = standardise(df_no75)
X_df_no75 = pd.DataFrame(X_df_no75, columns=df_no75.columns, index=df_no75.index)
print(X_df_no75.shape)

# Reduced dataset for code testing
X_df_red = X_df.iloc[:, 0:200]
print(X_df_red.shape)

(63, 48802)
(62, 48802)
(63, 200)


In [89]:
# retriving saved results:
import pickle

with open('neonatal_preprocessed.pkl', 'wb') as f:
    pickle.dump([gene_df, X_df, X_df_no75, X_df_red, labels, labels_no75], f)

In [94]:
X_df_no75.to_csv('neonatal_data_processed.csv')


AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [93]:
pd.read_csv('neonatal_data_processed.csv', index_col=0)

,ILMN_2227757,ILMN_1683690,ILMN_1710146,ILMN_1822171,ILMN_1748473,ILMN_1822843,ILMN_1710170,ILMN_1704497,ILMN_1847638,ILMN_1772260,...,ILMN_1712859,ILMN_1747129,ILMN_1813344,ILMN_2292576,ILMN_1793445,ILMN_2065273,ILMN_1800873,ILMN_1666642,ILMN_1908209,ILMN_1670343
Con_001,0.479050,-0.138235,0.190287,0.417313,0.912011,0.315686,-0.739573,-0.127496,-0.931788,1.009489,...,-1.553490,0.216461,-0.283713,0.543436,-0.679029,-0.269263,1.260044,3.326382,4.054684,-1.112666
Con_017,-0.796642,0.171890,0.377130,1.360378,0.480996,1.673514,-0.649962,1.807527,-1.601138,1.506726,...,-0.739596,-0.698123,1.807757,-0.504326,-0.941835,0.970388,-0.260336,-1.397208,-0.561287,-0.247056
Con_021,0.842998,-0.018567,0.593805,0.619009,0.456604,0.620670,0.151053,-0.561857,-0.544413,-0.213574,...,-0.364277,-0.708119,0.645692,-0.081346,1.121258,0.433380,-0.036937,-0.868595,0.949872,-0.995543
Con_022b,-0.245093,-0.158461,-0.271325,-0.442166,-0.266965,0.917543,-0.106808,1.068957,-0.573158,-0.625975,...,0.053212,1.810735,0.668380,-1.900804,-0.050579,-0.947497,0.687527,0.666900,-0.418795,-0.199475
Con_028,-0.751618,-1.341659,-0.445607,0.631728,0.619020,-1.074588,-2.045337,1.627197,1.559456,2.419899,...,-0.640495,-0.876375,1.898924,0.160818,0.081459,-1.059083,0.088782,0.813555,3.000998,-0.457511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Inf_162a,0.873014,0.697756,0.749246,-0.965485,-0.391260,-0.724180,0.167512,-0.696712,0.367218,0.396779,...,-0.397311,-0.704787,-1.140515,-0.733574,0.004014,0.708858,0.554573,-0.805117,-0.568786,-0.259866
Inf_164,-1.430736,-1.547286,-0.729797,-1.178083,0.579584,2.053122,-0.735915,0.531102,0.022276,0.441554,...,1.287407,-1.017977,-1.372763,-1.702229,-0.144529,-0.487205,-0.552473,-0.702240,-0.392547,0.431890
Inf_191,-0.162548,0.861246,-0.161417,0.871583,-2.291614,-0.338083,-1.215061,0.802381,1.729189,-0.402100,...,0.915602,1.309296,-0.568351,-0.126550,-1.117040,-1.991873,-0.560613,0.509301,-0.409421,0.311107
Inf_198,-0.505859,0.522468,1.414972,0.867949,-0.322321,-0.961030,0.867943,-0.914677,0.161896,0.068037,...,-0.118282,-0.408255,-0.807612,0.703265,-0.571113,0.414201,-0.869030,-0.471314,1.251729,0.380649


In [ ]:
savetxt('labels_no75.csv', labels_no75, delimiter=',')


### De-duplicating based on ensembl_gene_id

NOTE: NOT YET DONE, MAY NEED TO GET THE IDS USING R

In [12]:
# identifying duplicate rows, and elimnating and replacing with max expression levels, across examples.

# drop duplicate 
test_column = 'ILMN_Gene'
full_data = raw_data.drop_duplicates(subset=['Probe_Id'], keep='first')

# get boolean series of duplicates based on test_column name
dd_data = full_data.duplicated(subset=[test_column], keep=False)

# get index of the duplicates - i.e. the ones where True
dup_index = dd_data[dd_data].index

# create new dataframe, dropping all rows in the duplicate index
clean_df = full_data.drop(index=dup_index)

clean_df.sort_values(by=['ILMN_Gene'])
# test to see if there are any further duplicate rows based on that test column
#clean_dups = clean_df.duplicated(subset=['ILMN_Gene'], keep='first')
#print(clean_dups)
#clean_dups[clean_dups].index

,NuID,Species,Source,Search_Key,Transcript,ILMN_Gene,Source_Reference_ID,RefSeq_ID,Unigene_ID,Entrez_Gene_ID,...,Inf_149,Inf_152,Inf_157,Inf_159a,Inf_162a,Inf_164,Inf_191,Inf_198,Inf_203,Fold change
25103,Ku8QhfS0n_hIOABXuE,Homo sapiens,RefSeq,ILMN_5579,ILMN_183371,7A5,NM_182762.2,NM_182762.2,NaN,346389.0,...,7.0267,6.9027,6.9288,6.9657,7.0408,6.9972,6.9992,6.9750,7.0849,NaN
29049,NlLnpDrjjo8PUQKCJI,Homo sapiens,RefSeq,ILMN_19974,ILMN_19974,A26B1,NM_207355.2,NM_207355.2,NaN,339010.0,...,7.0348,6.9117,7.0070,7.0041,7.1023,7.0822,7.0845,7.0491,7.0978,NaN
42813,Wje_nHHp916e8Akjro,Homo sapiens,RefSeq,ILMN_31305,ILMN_307810,A26C1B,NM_001099771.1,NM_001099771.1,NaN,728378.0,...,7.0592,7.0157,7.0641,6.9264,7.0590,7.1122,7.0724,7.1493,7.0153,NaN
47175,ZfoDn4IUQp4Auf56LU,Homo sapiens,RefSeq,ILMN_8023,ILMN_8023,A2M,NM_000014.4,NM_000014.4,NaN,2.0,...,6.9565,7.0456,7.0070,7.0510,6.9608,7.0524,6.9637,6.9195,7.1239,NaN
7232,97DZJV_gpQpKINl0Ec,Homo sapiens,RefSeq,ILMN_17375,ILMN_17375,A2ML1,NM_144670.2,NM_144670.2,NaN,144568.0,...,7.0741,7.1184,7.0750,7.0510,6.9889,6.9835,7.0724,7.0491,7.0849,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31612,opLIq5393IogH5SiEI,Homo sapiens,RefSeq,ILMN_24644,ILMN_24644,ZUFSP,NM_145062.1,NM_145062.1,NaN,221302.0,...,8.1171,7.7784,7.7894,8.1040,8.1978,7.8296,7.7331,7.8357,7.8272,NaN
252,05KVyf6Big_oj1KLSo,Homo sapiens,RefSeq,ILMN_22056,ILMN_22056,ZW10,NM_004724.2,NM_004724.2,NaN,9183.0,...,7.9831,8.1483,7.9581,7.2240,7.5738,7.8844,7.7331,7.6729,7.5341,-0.3903
15920,EVdUtxU1JfTj09e7Qo,Homo sapiens,RefSeq,ILMN_4827,ILMN_4827,ZXDB,NM_007157.3,NM_007157.3,NaN,158586.0,...,7.1934,7.1242,7.2487,7.0580,7.0641,7.1840,7.1504,7.3298,7.1376,NaN
24628,KoL5XxK6hf_7w3v91U,Homo sapiens,RefSeq,ILMN_16610,ILMN_16610,ZYG11B,NM_024646.1,NM_024646.1,NaN,79699.0,...,8.0346,8.6077,8.3911,8.4578,8.6710,8.2491,8.1199,8.1083,8.6916,NaN


In [13]:
clean_dups = clean_df.dropna(subset=['Transcript']).duplicated(subset=['Transcript'], keep=False)
dup_index_2 = clean_dups[clean_dups].index
clean_df_2 = clean_df.drop(index=dup_index_2)
clean_df_2

,NuID,Species,Source,Search_Key,Transcript,ILMN_Gene,Source_Reference_ID,RefSeq_ID,Unigene_ID,Entrez_Gene_ID,...,Inf_149,Inf_152,Inf_157,Inf_159a,Inf_162a,Inf_164,Inf_191,Inf_198,Inf_203,Fold change
1,0_9dKJSRdZ6nRKsR5o,Homo sapiens,RefSeq,ILMN_42795,ILMN_309061,DCDC2B,NM_001099434.1,NM_001099434.1,NaN,149069.0,...,7.0826,7.0923,7.1350,6.9426,7.0748,6.9416,7.0845,7.0644,7.1239,0.0169
2,0_9Zd7UXqV4FRJ5XdE,Homo sapiens,RefSeq,ILMN_35279,ILMN_35279,LOC649978,XM_939064.1,XM_939064.1,NaN,649978.0,...,7.0010,7.1420,7.1856,7.1013,7.1069,7.0127,7.0489,7.1493,7.1517,0.0317
3,0_CoxIt3fv09UVwOo0,Homo sapiens,Unigene,ILMN_85446,ILMN_85446,HS.302418,Hs.302418,NaN,Hs.302418,NaN,...,7.0177,7.0861,7.0184,7.0380,6.9952,6.9835,7.0963,7.0961,7.0978,-0.0050
4,0_DHz7_krTURH3lVHk,Homo sapiens,RefSeq,ILMN_26906,ILMN_26906,GIMAP4,NM_018326.2,NM_018326.2,NaN,55303.0,...,12.1522,11.0532,12.8357,12.8616,11.9632,12.6279,10.6621,12.0104,12.0981,-0.6525
5,0_dJiKAH3o6kidUsOo,Homo sapiens,Unigene,ILMN_135414,ILMN_135414,HS.583233,Hs.583233,NaN,Hs.583233,NaN,...,7.0665,6.9750,6.9514,7.0041,6.9754,7.1466,6.9992,6.9608,6.9240,-0.0128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48791,ZzoXZ08Fvp.Kjq6aYk,Homo sapiens,Unigene,ILMN_127410,ILMN_127410,HS.575229,Hs.575229,NaN,Hs.575229,NaN,...,6.9830,7.1989,7.1112,7.6210,7.5831,7.1840,7.2971,7.1131,7.0041,NaN
48792,ZzQeInuQpqo7l66ISI,Homo sapiens,Unigene,ILMN_106946,ILMN_106946,HS.543340,Hs.543340,NaN,Hs.543340,NaN,...,7.0348,7.1355,7.0298,7.0171,7.1800,7.0672,7.1234,6.9608,7.0733,NaN
48794,ZZQvowA1A0rdICPUkk,Homo sapiens,RefSeq,ILMN_24212,ILMN_24212,CASP8AP2,NM_012115.2,NM_012115.2,NaN,9994.0,...,7.0826,7.1113,7.0518,7.0041,7.0160,6.9972,7.1369,7.0338,7.0494,NaN
48798,ZztzUERXUExGBETFFU,Homo sapiens,RefSeq,ILMN_14674,ILMN_14674,NPY6R,NR_002713.1,NR_002713.1,NaN,NaN,...,7.0010,7.0157,6.9841,6.9733,7.0813,7.0127,6.9264,7.0644,6.9361,NaN
